In [1]:
import sys
sys.path.append('..')

from misc import load, load_class_labels

dataset_root = "../dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"
val_restores_dir = dataset_root + "val_set_restored/"
val_prep_dit = dataset_root + "val_set_preprocessed/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train_info_cleaned.csv', 1, sample_rate)
test_info = load(dataset_root + 'labels/test.csv')
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values
test_image_names = test_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
test_labels = test_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

2024-01-15 11:07:14.512409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 11:07:14.512463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 11:07:14.516017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 11:07:14.522917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 11:07:15.378461: W tensorflow/compiler/tf2

In [3]:
from cnn_classification import build_finetune_network, train_network
from tensorflow import keras
from keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input

save_path = "./tuned_models/efficientnet_v2_noise_extended"

In [ ]:
extension = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(251, activation='softmax')])

loss = 'categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy', keras.metrics.TopKCategoricalAccuracy(3)]

cut_layer = 'top_dropout'
non_trainable_cut_layer = 'block5e_add'

extended_model = build_finetune_network(EfficientNetV2B0, extension, cut_layer,
                                        optimizer, metrics, loss, non_trainable_cut_layer=non_trainable_cut_layer)

In [4]:
extended_model = keras.models.load_model(save_path)

2024-01-15 11:07:26.459474: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-15 11:07:26.563691: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-15 11:07:26.563882: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
extended_model, train_loss, train_accuracy, val_loss, val_accuracy = train_network(extended_model,
                                                                   save_path,
                                                                   train_info,
                                                                   val_info,
                                                                   train_dir,
                                                                   epochs=5,
                                                                   augment=True,
                                                                   strong_augment=True,
                                                                   batch_size=64,
                                                                   preprocess_input=preprocess_input)

In [5]:
from tensorflow import keras
from cnn_classification import predict

predictions, names = predict(extended_model, val_dir, 128, preprocess_input=preprocess_input)

2024-01-15 11:07:37.486788: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.
2024-01-15 11:07:37.699769: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.
2024-01-15 11:07:38.964279: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-01-15 11:07:38.964331: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-01-15 11:07:38.964383: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-01-15 11:07:41.838520: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


3/3 [==============================] - 4s 1s/step


In [6]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np




ordered_indices = np.argsort(names)
predictions = np.array(predictions)
ordered_names = np.argsort(test_image_names)

predictions_logit = np.argmax(predictions[ordered_indices], axis=1)

filtered_info = test_info[test_info['filename'].isin(names)]
filtered_info.sort_values(by='filename', inplace=True)
labels = filtered_info['label'].values

ordered_y = labels


# cm = confusion_matrix(ordered_y, predictions_logit)
# disp = ConfusionMatrixDisplay(cm)
# disp.plot()
# plt.show()

In [7]:
from sklearn.metrics import classification_report

print(classification_report(ordered_y, predictions_logit, target_names=class_labels))

                               precision    recall  f1-score   support

                      macaron       0.94      0.87      0.91        55
                      beignet       0.85      0.82      0.83        61
                      cruller       0.82      0.58      0.68        53
                  cockle_food       0.55      0.71      0.62        51
                       samosa       0.57      0.80      0.67        41
                     tiramisu       0.67      0.58      0.62        52
                      tostada       0.63      0.60      0.61        57
                     moussaka       0.33      0.57      0.42        54
                     dumpling       0.58      0.61      0.59        49
                      sashimi       0.70      0.72      0.71        46
                        knish       0.51      0.38      0.43        48
                    croquette       0.38      0.64      0.48        47
                     couscous       0.47      0.47      0.47        45
     

In [8]:
import os
import pandas as pd
os.makedirs("predictions", exist_ok=True)

pred_file = "predictions/efficientnet_v2_extended_standard"
np.save(pred_file, predictions)
df = pd.DataFrame({
    'filename': names
})
df.to_csv(pred_file + "_names.csv")

In [9]:
def top_k(k = 1):
    ohe = keras.utils.to_categorical(ordered_y, num_classes=251)
    m = keras.metrics.TopKCategoricalAccuracy(k)
    ordered_pred = predictions[ordered_indices]
    m.update_state(ohe, ordered_pred)
    return m.result().numpy()

In [10]:
top_k(1)

0.59629816

In [11]:
top_k(3)

0.7874771